# Explore here

In [1]:
# Impportamos Librerias necesarias
import pandas as pd
import sqlite3
import json
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pickle import dump

# Datos

In [2]:
# Cargamos los datasets
df_movies = pd.read_csv('../data/raw/tmdb_5000_movies.csv')
df_credits = pd.read_csv('../data/raw/tmdb_5000_credits.csv')

In [3]:
# Tamaño de los datasets
df_movies.shape, df_credits.shape

((4803, 20), (4803, 4))

# Base de Datos SQL

In [4]:
# Creamos la base de datos SQLite
conn = sqlite3.connect('movies_database.db')

In [5]:
# Almacenar en tablas separadas
df_movies.to_sql('movies', conn, index=False, if_exists='replace')
df_credits.to_sql('credits', conn, index=False, if_exists='replace')

4803

In [6]:
# Unimos las tablas con SQL
query = """
SELECT m.id AS movie_id,
       m.title,
       m.overview,
       m.genres,
       m.keywords,
       c.cast,
       c.crew
FROM movies m
INNER JOIN credits c
ON m.title = c.title;
"""

data_unida = pd.read_sql_query(query, conn)
conn.close()

Para unir los datasets con la columna 'title' se pudo usar este código


'''''
df_unido = pd.merge(
    df_credits,
    df_movies,
    on='title',
    how='inner',
    validate='m:m')
'''

In [7]:
# Eliminamos columnas duplicadas
data_unida = data_unida.loc[:, ~data_unida.columns.duplicated()]

# Procesar datos Json

In [8]:
# Función para transformar las columnas JSON
def procesar_json(columna, clave='name', limite=None):
    def convertir(x):
        try:
            datos = literal_eval(x) if isinstance(x, str) else x
            return [item[clave] for item in datos][:limite]
        except:
            return []
    return convertir

In [9]:
# Procesamos las columnas JSON
data_unida['genres'] = data_unida['genres'].apply(procesar_json('genres'))
data_unida['keywords'] = data_unida['keywords'].apply(procesar_json('keywords'))
data_unida['cast'] = data_unida['cast'].apply(procesar_json('cast', limite=3))

In [10]:
# Procesamos crew para obtener director
def obtener_director(x):
    try:
        crew = literal_eval(x) if isinstance(x, str) else x
        return [m['name'] for m in crew if m['job'] == 'Director']
    except:
        return []

data_unida['crew'] = data_unida['crew'].apply(obtener_director)

In [11]:
# Función para eliminar espacios entre palabras
def quitar_espacios(lista):
    return [texto.replace(" ", "") for texto in lista]

for columna in ['genres', 'cast', 'crew', 'keywords']:
    data_unida[columna] = data_unida[columna].apply(quitar_espacios)

In [12]:
# Creamos la columna tags
data_unida['tags'] = (
    data_unida['overview'].fillna('').apply(lambda x: [x]) +
    data_unida['genres'] +
    data_unida['keywords'] +
    data_unida['cast'] +
    data_unida['crew'])

data_unida['tags'] = data_unida['tags'].apply(lambda x: ' '.join(x))

In [13]:
# DataFrame final
df = data_unida[['movie_id', 'title', 'tags']]

In [14]:
df.duplicated().sum()

0

In [15]:
df.isnull().sum()

movie_id    0
title       0
tags        0
dtype: int64

# Modelo KNN

In [16]:
# Vectorización TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['tags'])

In [17]:
# Calcular similitud coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [18]:
# Mapeo de títulos a índices
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [19]:
# Función de recomendación
def recomendar_pelicula(titulo):
    try:
        # Obtener índice
        idx = indices[titulo]
        
        # Obtener puntuaciones de similitud
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Ordenar por similitud
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Obtener top 5 (excluyendo la misma película)
        top_peliculas = sim_scores[1:6]
        
        # Obtener índices
        peliculas_indices = [i[0] for i in top_peliculas]
        
        # Mostrar resultados
        print(f"\nRecomendaciones para '{titulo}':")
        for i, indice in enumerate(peliculas_indices, 1):
            print(f"{i}. {df.iloc[indice]['title']}")
            
    except KeyError:
        print(f"\nError: No se encontró la película '{titulo}' en la base de datos.")

# Ejemplo de Uso

In [20]:
recomendar_pelicula("Avatar")


Recomendaciones para 'Avatar':
1. Aliens
2. Falcon Rising
3. Battle: Los Angeles
4. Apollo 18
5. Star Trek Into Darkness


In [21]:
recomendar_pelicula("The Godfather")


Recomendaciones para 'The Godfather':
1. The Godfather: Part II
2. The Godfather: Part III
3. The Master of Disguise
4. Easy Money
5. Mickey Blue Eyes
